In [440]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [441]:
df_1 = pd.read_csv('/Users/camil/Documents/iron_hack_bootcamp/week_5/2_weeks_project/two_week_project/data/clean/all_clients.csv')
df_2 = pd.read_csv('/Users/camil/Documents/iron_hack_bootcamp/week_5/2_weeks_project/two_week_project/data/clean/test_filter.csv')
df_3 = pd.read_csv('/Users/camil/Documents/iron_hack_bootcamp/week_5/2_weeks_project/two_week_project/data/clean/control_filter.csv')

In [442]:
df_2.head()

,Unnamed: 0,client_id,visitor_id,visit_id,process_step,date_time,variation
0,0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test
1,1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test
2,2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test
3,3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test
4,4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test


In [448]:
def calculate_error_rate(df, step_column, client_column, time_column):
    import pandas as pd

    # Map the process steps to numeric values for easier comparison
    step_mapping = {
        'start': 0,
        'step_1': 1,
        'step_2': 2,
        'step_3': 3,
        'confirm': 4
    }

    # Map process steps to numeric for comparison
    df['process_step_num'] = df[step_column].map(step_mapping)
    
    # Sort by client_id and time to ensure chronological order
    df = df.sort_values(by=[client_column, time_column], ascending=[True, True])
    
    # Filter the DataFrame to only include rows between the last 'start' and the last 'confirm'
    filtered_data = []
    for client_id, group in df.groupby(client_column):
        # Find the indices of the last 'start' and 'confirm' for each client
        last_start_idx = group[group[step_column] == 'start'].index.max()
        last_confirm_idx = group[group[step_column] == 'confirm'].index.max()

        # Include only rows between the last start and confirm, inclusive
        if pd.notna(last_start_idx) and pd.notna(last_confirm_idx) and last_start_idx < last_confirm_idx:
            filtered_data.append(group.loc[last_start_idx:last_confirm_idx])

    # Concatenate filtered results (handle cases with no valid data)
    if filtered_data:
        filtered_df = pd.concat(filtered_data)
    else:
        filtered_df = pd.DataFrame(columns=df.columns)

    # Add a new column to store error steps
    filtered_df['errors_step'] = None

    # Initialize error tracking variables
    errors = 0
    transitions = 0
    step_errors = {step_mapping[step]: 0 for step in step_mapping}  # Track errors per step dynamically
    step_transitions = {step_mapping[step]: 0 for step in step_mapping}  # Track transitions per step dynamically

    # Group the filtered data by client_id
    grouped = filtered_df.groupby(client_column)

    for _, client_group in grouped:
        # Reset the index for each client group
        client_steps = client_group.reset_index()
        last_valid_step_num = None

        for i in range(len(client_steps)):
            current_step_num = client_steps.loc[i, "process_step_num"]
            current_step_name = client_steps.loc[i, step_column]

            # Ignore transitions starting with 'start'
            if last_valid_step_num == 0:  # Last step was 'start'
                last_valid_step_num = current_step_num
                continue
            
            if last_valid_step_num is not None:
                transitions += 1
                step_transitions[last_valid_step_num] += 1

                # Repeated step error
                if current_step_num == last_valid_step_num:
                    step_errors[current_step_num] += 1
                    errors += 1
                    filtered_df.loc[client_steps.loc[i, 'index'], 'errors_step'] = current_step_name

                # Backward step error
                elif current_step_num < last_valid_step_num:
                    step_errors[current_step_num] += 1
                    errors += 1
                    filtered_df.loc[client_steps.loc[i, 'index'], 'errors_step'] = current_step_name

            # Update last valid step
            last_valid_step_num = current_step_num

    # Calculate global error rate
    error_rate = errors / transitions if transitions > 0 else 0

    # Calculate error rate per step
    error_rate_per_step = {
        step: (step_errors.get(step_mapping[step], 0) / step_transitions.get(step_mapping[step], 1)
               if step_transitions.get(step_mapping[step], 0) > 0 else 0)
        for step in step_mapping
    }

    return error_rate, error_rate_per_step, filtered_df


In [464]:
def calculate_completion_rate_final(df, step_column, client_column, time_column):
    import pandas as pd

    # Map the process steps to numeric values for easier comparison
    step_mapping = {
        'start': 0,
        'step_1': 1,
        'step_2': 2,
        'step_3': 3,
        'confirm': 4
    }

    # Map process steps to numeric for comparison
    df['process_step_num'] = df[step_column].map(step_mapping)
    
    # Sort by client_id and time to ensure chronological order
    df = df.sort_values(by=[client_column, time_column], ascending=[True, True])
    
    # Count clients with 'confirm' as the last step
    last_steps = df.groupby(client_column).tail(1)
    clients_with_confirm = last_steps[last_steps[step_column] == 'confirm'][client_column].nunique()
    total_clients = df[client_column].nunique()
    global_completion_rate = clients_with_confirm / total_clients if total_clients > 0 else 0

    # Initialize completion tracking variables
    completions = 0
    transitions = 0
    step_completions = {}  # Track completions per step dynamically
    step_transitions = {}  # Track transitions per step dynamically

    # Group the DataFrame by client_id
    grouped = df.groupby(client_column)

    for _, client_group in grouped:
        client_steps = client_group.reset_index()
        last_valid_step_num = None

        for i in range(len(client_steps)):
            current_step_num = client_steps.loc[i, "process_step_num"]
            
            # Dynamically initialize transitions and completions for the step
            if current_step_num not in step_transitions:
                step_transitions[current_step_num] = 0
                step_completions[current_step_num] = 0

            if last_valid_step_num is not None:
                transitions += 1
                step_transitions[last_valid_step_num] += 1

                # Success for the previous step (transition to a new step)
                if current_step_num > last_valid_step_num:
                    step_completions[last_valid_step_num] += 1
                    completions += 1

            last_valid_step_num = current_step_num

    # Calculate completion rate per step
    completion_rate_per_step = {
        step: (step_completions.get(step_mapping[step], 0) / step_transitions.get(step_mapping[step], 1)
               if step_transitions.get(step_mapping[step], 0) > 0 else 0)
        for step in step_mapping
    }

    return global_completion_rate, completion_rate_per_step

In [ ]:
def calculate_completion_rate_with_column(df, step_column, client_column, time_column):
    import pandas as pd

    # Map the process steps to numeric values for easier comparison
    step_mapping = {
        'start': 0,
        'step_1': 1,
        'step_2': 2,
        'step_3': 3,
        'confirm': 4
    }

    # Map process steps to numeric for comparison
    df['process_step_num'] = df[step_column].map(step_mapping)
    
    # Sort by client_id and time to ensure chronological order
    df = df.sort_values(by=[client_column, time_column], ascending=[True, True])
    
    # Count clients with 'confirm' as the last step
    last_steps = df.groupby(client_column).tail(1)
    clients_with_confirm = last_steps[last_steps[step_column] == 'confirm'][client_column].nunique()
    total_clients = df[client_column].nunique()
    global_completion_rate = clients_with_confirm / total_clients if total_clients > 0 else 0

    # Add a column to track completion steps
    df['completion_step'] = None

    # Initialize completion tracking variables
    completions = 0
    transitions = 0
    step_completions = {}  # Track completions per step dynamically
    step_transitions = {}  # Track transitions per step dynamically

    # Group the DataFrame by client_id
    grouped = df.groupby(client_column)

    for _, client_group in grouped:
        client_steps = client_group.reset_index()
        last_valid_step_num = None

        for i in range(len(client_steps)):
            current_step_num = client_steps.loc[i, "process_step_num"]
            
            # Dynamically initialize transitions and completions for the step
            if current_step_num not in step_transitions:
                step_transitions[current_step_num] = 0
                step_completions[current_step_num] = 0

            if last_valid_step_num is not None:
                transitions += 1
                step_transitions[last_valid_step_num] += 1

                # Success for the previous step (transition to a new step)
                if current_step_num > last_valid_step_num:
                    step_completions[last_valid_step_num] += 1
                    completions += 1
                    # Record the completed step in the column
                    df.loc[client_steps.loc[i, 'index'], 'completion_step'] = list(step_mapping.keys())[
                        list(step_mapping.values()).index(last_valid_step_num)
                    ]

            last_valid_step_num = current_step_num

    # Calculate completion rate per step
    completion_rate_per_step = {
        step: (step_completions.get(step_mapping[step], 0) / step_transitions.get(step_mapping[step], 1)
               if step_transitions.get(step_mapping[step], 0) > 0 else 0)
        for step in step_mapping
    }

    return global_completion_rate, completion_rate_per_step, df


In [451]:
error_rate_control, error_rate_control_per_step, df_result_control = calculate_error_rate(df_3, step_column='process_step', client_column='client_id', time_column='date_time')
error_rate_test, error_rate_test_per_step, df_result_test = calculate_error_rate(df_2, step_column='process_step', client_column='client_id', time_column='date_time')


In [452]:
print("errors rate for control group: ", round(error_rate_control * 100, 2), '%')
print("errors rate for test group: ", round(error_rate_test * 100, 2), '%')

errors rate for control group:  27.91 %
errors rate for test group:  37.19 %


In [453]:
print("Control Error Rate Per Step:", error_rate_control_per_step)
print("Test Error Rate Per Step:", error_rate_test_per_step)


Control Error Rate Per Step: {'start': 0, 'step_1': 0.13147410358565736, 'step_2': 0.16858638743455498, 'step_3': 0.08397790055248619, 'confirm': 0.9204545454545454}
Test Error Rate Per Step: {'start': 0, 'step_1': 0.11904761904761904, 'step_2': 0.0898936170212766, 'step_3': 0.05889014722536806, 'confirm': 0.9816326530612245}


In [454]:
# Données fournies
error_rates_control = {'start': 0, 'step_1': 0.13147410358565736, 'step_2': 0.16858638743455498,
                       'step_3': 0.08397790055248619, 'confirm': 0.9204545454545454}
error_rates_test = {'start': 0, 'step_1': 0.11904761904761904, 'step_2': 0.0898936170212766,
                    'step_3': 0.05889014722536806, 'confirm': 0.9816326530612245}
total_transitions_control = 119930
total_transitions_test = 150879

# Calcul des erreurs totales sans 'confirm'
total_errors_control_excl_confirm = (
    error_rates_control['step_1'] * total_transitions_control +
    error_rates_control['step_2'] * total_transitions_control +
    error_rates_control['step_3'] * total_transitions_control
)
total_errors_test_excl_confirm = (
    error_rates_test['step_1'] * total_transitions_test +
    error_rates_test['step_2'] * total_transitions_test +
    error_rates_test['step_3'] * total_transitions_test
)

# Calcul des taux globaux d'erreur sans 'confirm'
global_error_rate_control_excl_confirm = total_errors_control_excl_confirm / total_transitions_control
global_error_rate_test_excl_confirm = total_errors_test_excl_confirm / total_transitions_test

print("Errors rate for control group: ", round(global_error_rate_control_excl_confirm * 100, 2), '%')
print("Errors rate for test group: ", round(global_error_rate_test_excl_confirm * 100, 2), '%')


Errors rate for control group:  38.4 %
Errors rate for test group:  26.78 %


In [475]:
global_completion_rate_control, completion_rate_per_step_control= calculate_completion_rate_final(df_3, step_column='process_step', client_column='client_id', time_column='date_time')
global_completion_rate_test, completion_rate_per_step_test= calculate_completion_rate_final(df_2, step_column='process_step', client_column='client_id', time_column='date_time')

In [476]:
print("Completion rate for control group: ", round(global_completion_rate_control * 100, 2), '%')
print("Completion rate for test group: ", round(global_completion_rate_test * 100, 2), '%')

Completion rate for control group:  58.01 %
Completion rate for test group:  67.29 %


In [477]:
print("Control Completion Rate Per Step:", completion_rate_per_step_control)
print("Test Completion Rate Per Step:", completion_rate_per_step_test)

Control Completion Rate Per Step: {'start': 0.62513682006377, 'step_1': 0.8219582407171228, 'step_2': 0.8522980279898219, 'step_3': 0.7145320656226697, 'confirm': 0.0}
Test Completion Rate Per Step: {'start': 0.6747548971452149, 'step_1': 0.7489895996120063, 'step_2': 0.8034780893520497, 'step_3': 0.7417383613411594, 'confirm': 0.0}


In [478]:
df_profile = pd.read_csv('/Users/camil/Documents/iron_hack_bootcamp/week_5/2_weeks_project/two_week_project/data/raw/Client_Profiles.txt')

In [ ]:

from scipy.stats import chi2_contingency

# Define total transitions for each group
total_transitions_control = 119930
total_transitions_test = 150879

# Error rates for control and test groups
error_rates_control = {'step_1': 0.13147410358565736, 'step_2': 0.16858638743455498,
                       'step_3': 0.08397790055248619, 'confirm': 0.9204545454545454}
error_rates_test = {'step_1': 0.11904761904761904, 'step_2': 0.0898936170212766,
                    'step_3': 0.05889014722536806, 'confirm': 0.9816326530612245}

# Calculate the number of errors for each step
error_counts_control = {step: int(rate * total_transitions_control) for step, rate in error_rates_control.items()}
error_counts_test = {step: int(rate * total_transitions_test) for step, rate in error_rates_test.items()}

# Perform Chi-2 test for each step
step_results_chi2 = []

for step in error_counts_control.keys():
    # Control and test counts
    control_errors = error_counts_control[step]
    test_errors = error_counts_test[step]

    # Calculate successes (non-errors)
    control_successes = total_transitions_control - control_errors
    test_successes = total_transitions_test - test_errors

    # Create contingency table
    contingency_table = [[control_errors, control_successes],
                         [test_errors, test_successes]]

    # Perform Chi-2 test
    chi2_stat, p_value, _, _ = chi2_contingency(contingency_table, correction=False)

    # Append results
    step_results_chi2.append({
        'step': step,
        'control_errors': control_errors,
        'test_errors': test_errors,
        'chi2_stat': chi2_stat,
        'p_value': p_value
    })

# Convert results to a DataFrame
step_results_chi2_df = pd.DataFrame(step_results_chi2)

step_results_chi2_df.head()



,step,control_errors,test_errors,chi2_stat,p_value
0,step_1,15767,17961,94.621808,2.304652e-22
1,step_2,20218,13563,3789.391863,0.000000e+00
2,step_3,10071,8885,645.921309,1.722465e-142
3,confirm,110390,148107,5761.960488,0.000000e+00


In [482]:
df_result = pd.concat([df_result_control, df_result_test], axis=0)


In [484]:
profile_errors= df_result.merge(df_profile, on='client_id', how='inner')

In [485]:
profile_errors['age_group'] = pd.cut(
    profile_errors['clnt_age'], 
    bins=[0, 18, 25, 35, 50, 65, 100], 
    labels=['<18', '18-25', '26-35', '36-50', '51-65', '65+']
)
profile_errors['tenure_group'] = pd.cut(
    profile_errors['clnt_tenure_mnth'], 
    bins=[0, 12, 36, 60, 120, 240], 
    labels=['<1 year', '1-3 years', '3-5 years', '5-10 years', '10+ years']
)

In [489]:
profile_errors.head()

,client_id,visitor_id,visit_id,process_step,date_time,variation,process_step_num,errors_step,completion_step,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,age_group
0,16549,536420175_30932891358,753427569_18850244409_814756,start,2017-05-04 13:32:04,Control,0,None,None,24.0,294.0,55.5,F,3.0,314799.02,4.0,7.0,51-65
1,16549,536420175_30932891358,753427569_18850244409_814756,step_1,2017-05-04 13:32:20,Control,1,None,start,24.0,294.0,55.5,F,3.0,314799.02,4.0,7.0,51-65
2,16549,784001246_8842193453,987504726_72578887699_966535,confirm,2017-05-13 13:17:10,Control,4,None,step_1,24.0,294.0,55.5,F,3.0,314799.02,4.0,7.0,51-65
3,29595,343737541_19063774853,253828702_4091182456_53800,start,2017-05-10 14:21:34,Control,0,None,None,5.0,68.0,37.0,U,2.0,46514.58,3.0,6.0,36-50
4,29595,343737541_19063774853,253828702_4091182456_53800,step_1,2017-05-10 14:21:42,Control,1,None,start,5.0,68.0,37.0,U,2.0,46514.58,3.0,6.0,36-50


In [495]:
profile_errors.to_csv('/Users/camil/Documents/iron_hack_bootcamp/week_5/2_weeks_project/two_week_project/data/clean/profile_errors.csv', index=False)

In [ ]:
profile_errors = profile_errors.drop(columns=['Unnamed: 0'])

In [500]:
df_result.to_csv('/Users/camil/Documents/iron_hack_bootcamp/week_5/2_weeks_project/two_week_project/data/clean/df_result.csv', index=False)